In [ ]:
# load all datasets
import pandas as pd

foodcom_recipes = pd.read_csv("/kaggle/input/foodcom-recipes-and-reviews/recipes.csv")
foodcom_reviews = pd.read_csv("/kaggle/input/foodcom-recipes-and-reviews/reviews.csv")
foodcom_search_terms_and_tags = pd.read_csv("/kaggle/input/foodcom-recipes-with-search-terms-and-tags/recipes_w_search_terms.csv")

In [ ]:
#  1. From recipes.csv, only take the recipes that are not older than 10 years 
# (there is a column "DatePublished"). 
foodcom_recipes["DatePublished"] = foodcom_recipes["DatePublished"].apply(lambda x: int(str(x)[0:4]))
foodcom_recipes = foodcom_recipes[foodcom_recipes["DatePublished"] > 2012]
foodcom_recipes.head()

In [ ]:
# 2. Combine all recipes that are present in BOTH recipes.csv and recipes_w_search_terms.csv 
# as one dataset (we have RecipeID/id as primary key). Delete the unnecessary columns (see below). 
# There are columns Name, Description that are present in both datasets. I suggest we just take the 
# values from one dataset (doesn't matter which).
# The columns we DEFINITELY don't need:
# 1. From recipes.csv: AuthorId, AuthorName, Images
# 2. From recipes_w_search_terms.csv: serving_size, servings. 

foodcom_reviews.drop("AuthorId", axis=1, inplace=True)
foodcom_reviews.drop("AuthorName", axis=1, inplace=True)
foodcom_reviews.drop("DateSubmitted", axis=1, inplace=True)
foodcom_reviews.drop("DateModified", axis=1, inplace=True)
foodcom_reviews.head()

In [ ]:
foodcom_recipes.drop("AuthorId", axis=1, inplace=True)
foodcom_recipes.drop("AuthorName", axis=1, inplace=True)
foodcom_recipes.drop("Images", axis=1, inplace=True)
foodcom_recipes.drop("Name", axis=1, inplace=True)
foodcom_recipes.drop("Description", axis=1, inplace=True)
foodcom_recipes.head()

In [ ]:
foodcom_search_terms_and_tags.drop("serving_size", axis=1, inplace=True)
foodcom_search_terms_and_tags.drop("servings", axis=1, inplace=True)
foodcom_search_terms_and_tags.rename(columns={'id':'RecipeId'}, inplace=True)
foodcom_search_terms_and_tags.head()

In [ ]:
foodcom_combined_data = pd.merge(foodcom_reviews, foodcom_recipes, on='RecipeId')
foodcom_combined_data = pd.merge(foodcom_combined_data, foodcom_search_terms_and_tags, on='RecipeId')
print(len(foodcom_combined_data))
foodcom_combined_data.head()

In [ ]:
# 3. For the recipes that we now have, take only those that have at least 10 reviews in reviews.csv.
# For this I suggest using pandas SQL-like functions. Introduce a column "Average rating" which is 
# an average of all reviews for this recipe. And a column "Number of ratings" which is a number of 
# data records in reviews.csv for this recipe. 

foodcom_combined_data = foodcom_combined_data[foodcom_combined_data.groupby('RecipeId')['RecipeId'].transform('size') >= 10]
print(len(foodcom_combined_data))
foodcom_combined_data['number_of_ratings'] = foodcom_combined_data.groupby('RecipeId')['RecipeId'].transform('size')
foodcom_combined_data.sample(5)

In [ ]:
# this is inefficient but I cant come up with a cleaner solution
foodcom_combined_data['average_rating'] = foodcom_combined_data[["Rating"]].mean(axis=0)

for index,row in foodcom_combined_data.iterrows():
    all_rows_with_this_recipie_id = foodcom_combined_data[foodcom_combined_data["RecipeId"] == row["RecipeId"]]
    mean_of_all_ratings = all_rows_with_this_recipie_id['Rating'].mean()
    foodcom_combined_data.loc[index, "average_rating"] = mean_of_all_ratings
foodcom_combined_data.sample(5)

In [ ]:
# 4. So now we have a dataset with columns from recipes.csv and recipes_w_search_terms.csv, 
# and two additional columns: "Average rating" and "Number of ratings". Depending on the number of
# records that we now have, either leave the dataset as this, or sample a subset. 
print(len(foodcom_combined_data))

foodcom_combined_data.hist(column='Rating')
foodcom_combined_data.hist(column='number_of_ratings')
foodcom_combined_data.hist(column='average_rating')



In [ ]:
foodcom_combined_data.to_excel("/kaggle/working/foodcom_combined_data.xlsx")  